## GOLD TO API LAYER

### API Layer - Current Holdings


In [1]:
import pandas as pd
import json
from ETLTools import GlobalPath

In [2]:
# Define file paths using GlobalPath
current_holding_records_file_path = GlobalPath(
    "DATA/GOLD/Holdings/CurrentHoldings_data.csv"
)
stockprice_silver_file_path = GlobalPath(
    "DATA/SILVER/StockPrice/StockPrice_data.csv"
)
holdingshistory_gold_file_path = GlobalPath(
    "DATA/GOLD/Holdings/HoldingsHistory_data.csv"
)
current_holdings_api_file_path = GlobalPath("DATA/API/Holdings_data.json")

### Load and process holdings data from the GOLD layer


In [3]:
# Load holdings data from GOLD layer
df_current_holding = pd.read_csv(current_holding_records_file_path)
df_current_holding["datetime"] = pd.to_datetime(df_current_holding["datetime"])
print(
    f"Loaded GOLD Layer datetime data from: {current_holding_records_file_path.relative_path()}"
)

Loaded GOLD Layer datetime data from: DATA/GOLD/Holdings/CurrentHoldings_data.csv


In [4]:
# Group by scrip_name, symbol, exchange, and segment to calculate totals and min datetime
df_grouped = (
    df_current_holding.groupby(["scrip_name", "symbol", "exchange", "segment"])
    .agg(
        total_quantity=("quantity", "sum"),
        total_amount=("amount", "sum"),
        min_datetime=("datetime", "min"),
    )
    .reset_index()
)

# Calculate average price
df_grouped["avg_price"] = (
    df_grouped["total_amount"] / df_grouped["total_quantity"]
)

### Load and process stock prices from the SILVER layer


In [5]:
# Load stock prices data from SILVER layer
df_stockprice = pd.read_csv(stockprice_silver_file_path)
print(
    f"Loaded SILVER Layer stock price data from: {stockprice_silver_file_path.relative_path()}"
)

# Convert the 'date' column to datetime type
df_stockprice["date"] = pd.to_datetime(df_stockprice["date"])

# Extract the latest closing price for each symbol
df_stockprice["close_price"] = df_stockprice["close"]
idx = df_stockprice.groupby("symbol")["date"].idxmax()
df_stockprice = df_stockprice.loc[idx].reset_index(drop=True)

Loaded SILVER Layer stock price data from: DATA/SILVER/StockPrice/StockPrice_data.csv


### Merge data and calculate PnL


In [6]:
# Merge the grouped holdings data with the latest stock price data on symbol
df_grouped = pd.merge(
    df_grouped,
    df_stockprice[["symbol", "close_price"]],
    on="symbol",
    how="left",
)

# Calculate the close amount and PnL amount for each entry
df_grouped["close_amount"] = (
    df_grouped["close_price"] * df_grouped["total_quantity"]
)
df_grouped["pnl_amount"] = (
    df_grouped["close_amount"] - df_grouped["total_amount"]
)

# Round numerical columns to 2 decimal places
df_grouped = df_grouped.round(2)

In [7]:
# Prepare a list to hold the final result
current_holding = []
for _, row in df_grouped.iterrows():
    # Filter current holdings matching each grouped entry
    df_filtered = df_current_holding[
        (df_current_holding["scrip_name"] == row["scrip_name"])
        & (df_current_holding["symbol"] == row["symbol"])
        & (df_current_holding["exchange"] == row["exchange"])
        & (df_current_holding["segment"] == row["segment"])
    ]

    # Convert the row to a dictionary and add historical data
    row = row.to_dict()
    row["history"] = df_filtered.to_dict(orient="records")
    current_holding.append(row)

In [8]:
df_holdings = pd.read_csv(holdingshistory_gold_file_path)
df_holdings["date"] = pd.to_datetime(df_holdings["date"]).dt.date
print(
    f"Loaded GOLD Layer holdings data from: {holdingshistory_gold_file_path.relative_path()}"
)

Loaded GOLD Layer holdings data from: DATA/GOLD/Holdings/HoldingsHistory_data.csv


In [9]:
df_holdings_trands = (
    df_holdings.groupby("date")[
        [
            "holding_amount",
            "open_amount",
            "high_amount",
            "low_amount",
            "close_amount",
        ]
    ]
    .sum()
    .reset_index()
)
df_holdings_trands = df_holdings_trands.round(2)
df_holdings_trands = df_holdings_trands.rename(
    columns={
        col: col.replace("_amount", "") for col in df_holdings_trands.columns
    }
)
df_holdings_trands = df_holdings_trands[
    [
        "date",
        "open",
        "high",
        "low",
        "close",
        "holding",
    ]
]
df_holdings_trands = df_holdings_trands.sort_values(by=["date"]).reset_index(
    drop=True
)
holdings_trands = df_holdings_trands.to_dict(orient="records")

### Prepare final output dictionary and write to JSON file


In [10]:
# Create the final output dictionary for the API
output = {
    "invested_value": round(df_grouped["total_amount"].sum(), 2),
    "current_value": round(df_grouped["close_amount"].sum(), 2),
    "overall_pnl": round(df_grouped["pnl_amount"].sum(), 2),
    "current_holding": current_holding,
    "holdings_trands": holdings_trands,
}

# Write the result to a JSON file
with open(current_holdings_api_file_path, "w") as json_file:
    json.dump(output, json_file, indent=4, default=str)

print(f"Data written to {current_holdings_api_file_path.relative_path()}")

Data written to DATA/API/Holdings_data.json
